In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from PIL import Image

# Hyperparameters
batch_size = 4  # 큰 이미지를 처리하므로 배치 크기를 줄였습니다.
lr = 0.0002
num_epochs = 5
img_size = 512  # 이미지 크기를 512로 설정
channels = 1  # 흑백 이미지일 경우 1, 컬러 이미지일 경우 3
img_shape = (channels, img_size, img_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading
transform = transforms.Compose(
    [
        transforms.Resize((img_size, img_size)),  # 이미지 크기를 512x512로 조정
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)


# Custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_dir, gt_dir, transform=None):
        self.input_dir = input_dir
        self.gt_dir = gt_dir
        self.transform = transform
        self.input_images = sorted(os.listdir(input_dir))
        self.gt_images = sorted(os.listdir(gt_dir))

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        input_image = Image.open(
            os.path.join(self.input_dir, self.input_images[idx])
        ).convert("L")
        gt_image = Image.open(os.path.join(self.gt_dir, self.gt_images[idx])).convert(
            "L"
        )

        if self.transform:
            input_image = self.transform(input_image)
            gt_image = self.transform(gt_image)

        return input_image, gt_image


# Directories
input_dir = "./test_train_input"
gt_dir = "./test_train_gt"

# Data loading
custom_dataset = CustomDataset(input_dir, gt_dir, transform=transform)
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)


# Generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def down_block(
            in_channels, out_channels, kernel_size, stride, padding, normalize=True
        ):
            layers = [
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
            ]
            if normalize:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        def up_block(in_channels, out_channels, kernel_size, stride, padding):
            layers = [
                nn.ConvTranspose2d(
                    in_channels, out_channels, kernel_size, stride, padding
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
            ]
            return layers

        self.encoder = nn.Sequential(
            *down_block(
                channels, 64, 4, 2, 1, normalize=False
            ),  # [batch, 64, 256, 256]
            *down_block(64, 128, 4, 2, 1),  # [batch, 128, 128, 128]
            *down_block(128, 256, 4, 2, 1),  # [batch, 256, 64, 64]
            *down_block(256, 512, 4, 2, 1),  # [batch, 512, 32, 32]
            *down_block(512, 512, 4, 2, 1),  # [batch, 512, 16, 16]
            *down_block(512, 512, 4, 2, 1),  # [batch, 512, 8, 8]
            *down_block(512, 512, 4, 2, 1),  # [batch, 512, 4, 4]
            *down_block(512, 512, 4, 2, 1),  # [batch, 512, 2, 2]
            *down_block(512, 512, 4, 2, 1),  # [batch, 512, 1, 1]
        )

        self.decoder = nn.Sequential(
            *up_block(512, 512, 4, 2, 1),  # [batch, 512, 2, 2]
            *up_block(512, 512, 4, 2, 1),  # [batch, 512, 4, 4]
            *up_block(512, 512, 4, 2, 1),  # [batch, 512, 8, 8]
            *up_block(512, 512, 4, 2, 1),  # [batch, 512, 16, 16]
            *up_block(512, 512, 4, 2, 1),  # [batch, 512, 32, 32]
            *up_block(512, 256, 4, 2, 1),  # [batch, 256, 64, 64]
            *up_block(256, 128, 4, 2, 1),  # [batch, 128, 128, 128]
            *up_block(128, 64, 4, 2, 1),  # [batch, 64, 256, 256]
            nn.ConvTranspose2d(64, channels, 4, 2, 1),  # [batch, channels, 512, 512]
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# Discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def block(
            in_channels, out_channels, kernel_size, stride, padding, normalize=True
        ):
            layers = [
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
            ]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(channels, 64, 4, 2, 1, normalize=False),  # [batch, 64, 256, 256]
            *block(64, 128, 4, 2, 1),  # [batch, 128, 128, 128]
            *block(128, 256, 4, 2, 1),  # [batch, 256, 64, 64]
            *block(256, 512, 4, 2, 1),  # [batch, 512, 32, 32]
            *block(512, 512, 4, 2, 1),  # [batch, 512, 16, 16]
            *block(512, 512, 4, 2, 1),  # [batch, 512, 8, 8]
            *block(512, 512, 4, 2, 1),  # [batch, 512, 4, 4]
            nn.Conv2d(512, 1, 4, 1, 0),  # [batch, 1, 1, 1]
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x).view(-1, 1)


# 모델 초기화
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
criterion = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    for i, (input_imgs, gt_imgs) in enumerate(custom_dataloader):

        # Adversarial ground truths
        valid = torch.ones(input_imgs.size(0), 1, requires_grad=False).to(device)
        fake = torch.zeros(input_imgs.size(0), 1, requires_grad=False).to(device)

        # Configure input
        real_imgs = gt_imgs.to(device)
        input_imgs = input_imgs.to(device)

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_imgs = generator(input_imgs)

        # Loss measures generator's ability to fool the discriminator
        pred_fake = discriminator(gen_imgs)
        g_loss = criterion(pred_fake, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Real images
        pred_real = discriminator(real_imgs)
        real_loss = criterion(pred_real, valid)

        # Fake images
        pred_fake = discriminator(gen_imgs.detach())
        fake_loss = criterion(pred_fake, fake)

        # Total loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            f"[Epoch {epoch+1}/{num_epochs}] [Batch {i+1}/{len(custom_dataloader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]"
        )

[Epoch 1/5] [Batch 1/3] [D loss: 0.6984] [G loss: 0.8195]
[Epoch 1/5] [Batch 2/3] [D loss: 0.4961] [G loss: 1.0132]
[Epoch 1/5] [Batch 3/3] [D loss: 0.4357] [G loss: 2.4813]
[Epoch 2/5] [Batch 1/3] [D loss: 1.4248] [G loss: 0.0631]
[Epoch 2/5] [Batch 2/3] [D loss: 0.0001] [G loss: 16.5776]
[Epoch 2/5] [Batch 3/3] [D loss: 0.0008] [G loss: 9.6996]
[Epoch 3/5] [Batch 1/3] [D loss: 0.1128] [G loss: 2.5510]
[Epoch 3/5] [Batch 2/3] [D loss: 2.9027] [G loss: 0.0034]
[Epoch 3/5] [Batch 3/3] [D loss: 0.0000] [G loss: 14.8608]
[Epoch 4/5] [Batch 1/3] [D loss: 0.2348] [G loss: 13.8655]
[Epoch 4/5] [Batch 2/3] [D loss: 0.0323] [G loss: 11.9980]
[Epoch 4/5] [Batch 3/3] [D loss: 0.2173] [G loss: 8.5054]
[Epoch 5/5] [Batch 1/3] [D loss: 0.0478] [G loss: 2.4179]
[Epoch 5/5] [Batch 2/3] [D loss: 3.3070] [G loss: 0.0014]
[Epoch 5/5] [Batch 3/3] [D loss: 0.0312] [G loss: 11.4816]


In [14]:
import torch
from torchvision import transforms
from PIL import Image
import os

# Hyperparameters
img_size = 512  # 이미지 크기
channels = 1  # 흑백 이미지
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform 정의 (학습 시 사용한 것과 동일하게 설정)
transform = transforms.Compose(
    [
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

# 테스트 데이터 디렉토리
test_input_dir = "./test_test_input"
output_dir = "./test_output_images"
os.makedirs(output_dir, exist_ok=True)

# 모델 로드 (학습된 가중치를 로드했다고 가정)
generator = Generator().to(device)
generator.eval()  # 평가 모드로 전환

# 테스트 함수 정의
def generate_images(input_dir, output_dir):
    # 테스트 디렉토리의 모든 이미지 파일 로드
    test_images = sorted(os.listdir(input_dir))

    for idx, img_file in enumerate(test_images):
        input_path = os.path.join(input_dir, img_file)

        # 입력 이미지 로드 및 전처리
        input_image = Image.open(input_path).convert("L")
        input_tensor = transform(input_image).unsqueeze(0).to(device)  # 배치 차원 추가

        # 이미지 생성
        with torch.no_grad():
            gen_tensor = generator(input_tensor)

        # 생성된 이미지를 저장 ([-1, 1] -> [0, 1]로 변환)
        gen_tensor = (gen_tensor * 0.5 + 0.5).clamp(0, 1)  # 정규화 해제
        gen_image = gen_tensor.squeeze(0).squeeze(0).cpu().numpy()  # 배치, 채널 제거
        gen_image = (gen_image * 255).astype("uint8")  # [0, 1] -> [0, 255]

        # PIL 이미지로 변환하여 저장
        output_path = os.path.join(output_dir, f"generated_{idx+1}.png")
        Image.fromarray(gen_image).save(output_path)
        print(f"Generated image saved to {output_path}")

# 테스트 실행
generate_images(test_input_dir, output_dir)

Generated image saved to ./test_output_images/generated_1.png
Generated image saved to ./test_output_images/generated_2.png
Generated image saved to ./test_output_images/generated_3.png
Generated image saved to ./test_output_images/generated_4.png
Generated image saved to ./test_output_images/generated_5.png
Generated image saved to ./test_output_images/generated_6.png
Generated image saved to ./test_output_images/generated_7.png
Generated image saved to ./test_output_images/generated_8.png
Generated image saved to ./test_output_images/generated_9.png
Generated image saved to ./test_output_images/generated_10.png
